## 03 - Modelling

## Optional - Colab Setup

In [1]:
def restart_runtime():
    os.kill(os.getpid(), 9)

In [2]:
def setup_rapids():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    device_name = pynvml.nvmlDeviceGetName(handle)
    if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
        print("Wrong GPU - Restarting Runtime")
        restart_runtime()


    # clone RAPIDS AI rapidsai-csp-utils scripts repo
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git

    # install RAPIDS
    !bash rapidsai-csp-utils/colab/rapids-colab.sh 0.13


    # set necessary environment variables 
    dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
    sys.path

    # update pyarrow & modules 
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [3]:
def setup_conda():
    if not 'Miniconda3-4.5.4-Linux-x86_64.sh' in os.listdir():
        !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

    if not ('EPFL-Capstone-Project' in os.listdir()) and (os.getcwd().split('/')[-1] != 'EPFL-Capstone-Project'):
        !git clone https://github.com/helmigsimon/EPFL-Capstone-Project  
    os.chdir('EPFL-Capstone-Project')

    !conda env create -f environment.yml
    !conda activate exts-ml

In [4]:
def setup_drive():
    #Mounting Google Drive
    global drive
    from google.colab import drive
    drive.mount('/content/drive')

In [5]:
try:
    import sys,os
    
    setup_drive()

    #Setting up PyPi Packages
    !pip install geopandas sparse-dot-topn pdpipe category_encoders
    import geopandas as gpd
    import sparse_dot_topn.sparse_dot_topn as ct
    import pdpipe as pdp
    import category_encoders
    

    #Setting up Conda Packages
    setup_conda()
    
    #Initializing NLTK
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    
    #Setting up RAPIDS AI
    import pynvml
    setup_rapids()
    
    from cuml import UMAP
    
except ModuleNotFoundError as e:
    print(e)
    print('Not in colab environment, continuing to run locally')
    from umap import UMAP

No module named 'google.colab'
Not in colab environment, continuing to run locally


## Imports

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split,  StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
tqdm.pandas()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

In [8]:
from lib.transformers import *
from lib.pipelines import *
from lib.processing import save_to_pkl, load_from_pkl
from data.util.paths import DATA_PATH

## Setting up DataFrame

In [10]:
api_df = load_from_pkl('api',DATA_PATH)
extracted_df = load_from_pkl('extracted',DATA_PATH)
with np.load(os.path.join(DATA_PATH,'high_level_features_labelled.npz')) as data:
    image_embedding_df = pd.concat([pd.DataFrame(data[section]) for section in ('release_id','bitmap','features')],axis=1)
    image_embedding_df.columns = ['release_id', 'bitmap'] + ['feature_%s' % i for i in range(1,1281)]

In [11]:
df = api_df.merge(extracted_df,how='inner',on='release_id')
df = df.merge(image_embedding_df,how='inner',on='release_id')
del api_df, extracted_df, image_embedding_df

In [12]:
col_set = {
    'format': {
        'description': 'format_description_', 
        'name': 'format_name_', 
        'text': ('format_text_clean'),
        'quantity': ('format_quantity')
    },
    'geography': {
        'superregion': 'superregion_',
        'region': 'region_',
        'country': 'country_'
    },
    'timeperiod': {
        'period': 'period_',
        'era': 'era_'
    },
    'genre': 'genre_',
    'style': 'style_',
    'null': None,
    'indicator': lambda x: x.max() == 1 and x.min() == 0,
    'image_embedding': 'feature_'
}
column_store = ColumnStore()
column_store.fit(df,col_set)

## Reducing Dimensionality of Image Embeddings

In [ ]:
scaler = StandardScaler()
umap = UMAP(n_components=10)
image_embeddings_scaled = scaler.fit_transform(df.loc[:,sorted(list(column_store._image_embedding))])


In [15]:
image_embeddings_reduced = umap.fit_transform(image_embeddings_scaled)

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected row ('C') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


In [ ]:
df = pd.concat([
      df,
      pd.DataFrame(
          image_embeddings_reduced,
          columns = ['umap_%s' % i for i in range(image_embeddings_reduced.shape[1])]
      )],
      axis=1
)

In [ ]:
universal_processing_pipe = Pipeline([
    ('unpickle', Unpickler(['track_titles','genre','style','label','formats'])),
    ('make_market_value', ColumnCombiner('median','market_price','market_value')),
    ('remove_duplicates', DuplicateRemover('release_id')),
    ('remove_nulls', NullRemover('market_value')),
    ('count_standards', StandardCountEncoder('track_titles',DATA_PATH)),
    ('count_days_since_last_sale',LastSoldEncoder(feature='last_sold',new_feature='days_since_last_sale')),
    ('split_title', TitleSplitter()),
    ('clean_text', clean_text_pipe),
    ('make_format_columns',FormatEncoder())
])

In [ ]:
universal_processing_pipe.fit_transform(df)

## Preparing for Machine Learning

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(df.drop('market_value',axis=1),df.market_value)

In [ ]:
del df

Of the above columns, we will only handl ``running_time``, ``average_rating`` and ``units_for_sale``. The rest will not be necessary to handle for the purposes of the models we plan on building

In [ ]:
full_information_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [ ]:
record_store_transformer = ColumnTransformer(transformers=[
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [ ]:
record_store_ridge_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','label','community_have','community_want','formats','master_id','thumb_url',
    'release_url','artist','title','format_description','format_text_clean','format_text', 'no_of_days_since_last_sale'
    ]

In [ ]:
record_store_ridge_param_grid = dict(ridge__alpha=np.linspace(900,1000,10))

In [ ]:
record_store_ridge_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_ridge_removal_columns)),
    ('preprocessing',record_store_transformer),
    ('scaler', StandardScaler()),
    ('ridge', GridSearchCV(
        SGDRegressor(
            early_stopping=True,
            max_iter=1000,
            tol=0.001,
            n_iter_no_change=5,
            verbose=1
        ),
        param_grid=record_store_ridge_param_grid,
        cv=5
    ))
])

In [ ]:
record_store_ridge_pipe.fit(X_tr,np.log(y_tr))

In [ ]:
record_store_ridge_grid_search.best_params_

In [ ]:
record_store_ridge_grid_search.best_score_

In [ ]:
ridge_pred = record_store_ridge_grid_search.predict(X_te)

In [ ]:
r2_score(np.log(y_te),np.exp(ridge_pred))

In [ ]:
mean_absolute_error(np.log(y_te),np.exp(ridge_pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest_param_grid = {
    'n_estimators': tuple(range(50,350,50)),
}

In [ ]:
random_forest_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_ridge_removal_columns)),
    ('preprocessing',record_store_transformer),
    ('scaler', StandardScaler()),
    ('random_forest', GridSearchCV(
        RandomForestRegressor(
            random_state=0,
            n_jobs=-1,
            criterion='mae',
            verbose=50
        ),
        cv=5,
        param_grid=random_forest_param_grid,
        verbose=50
    ))
])

In [ ]:
random_forest_pipe.fit(X_tr,np.log(y_tr))

## Extremely Randomized Trees

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
extra_trees_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_ridge_removal_columns)),
    ('preprocessing',record_store_transformer),
    ('scaler', StandardScaler()),
    ('random_forest', GridSearchCV(
        ExtraTreesRegressor(
            random_state=0,
            n_jobs=4,
            verbose=50
        ),
        param_grid=random_forest_param_grid,
        verbose=50
    ))
])

In [ ]:
extra_trees_pipe.fit(X_tr,np.log(y_tr))

## Catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
record_store_catboost_removal_columns = (
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','community_have','community_want','formats','master_id','thumb_url',
    'release_url','artist','title','format_description','format_text_clean', 'no_of_days_since_last_sale'
)


In [ ]:
catboost_param_grid = {
        'depth': [4,7,10],
        'learning_rate' : np.logspace(-4,-1,5),
        'l2_leaf_reg': [1,4,9],
        'iterations': [100,300,500]
}

In [ ]:
catboost_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_catboost_removal_columns)),
    ('scaler', StandardScaler()),
    ('grid_search', GridSearchCV(
        CatBoostRegressor(
            random_state=0,
            cat_features=['year','format_text']),
        param_grid=catboost_param_grid,verbose=5)
    )
])

In [ ]:
catboost_pipe.fit(X_tr,np.log(y_tr))